<a href="https://colab.research.google.com/github/shaunck96/Pennsylvania_fire_risk_bot/blob/main/Outage_and_Fire_Risk_Streamlit_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_fire_risk_data_by_region():
  historic_fire_risk_data = pd.DataFrame()
  for i in range(1,13,1):
    # URL of the page
    url = "https://data.publicopiniononline.com/fires/?page="+str(i)+"&anchor=#firetable"

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the table
        table = soup.find('table')

        # Extract the rows from the table
        rows = table.find_all('tr')

        # Extract the header and data from the rows
        headers = [header.text for header in rows[0].find_all('th')]
        data = [[cell.text for cell in row.find_all('td')] for row in rows[1:]]

        # Create a DataFrame from the data
        df = pd.DataFrame(data, columns=headers)

        historic_fire_risk_data = pd.concat([historic_fire_risk_data, df])

    else:
        print(f"Failed to fetch the page. Status code: {response.status_code}")

  return historic_fire_risk_data

historic_fire_risk_data = fetch_fire_risk_data_by_region()
historic_fire_risk_data.head(10)

Failed to fetch the page. Status code: 500


,"Name (County, State)",County,State,Fire Discoverd Date,% Contained,Size (Acres),Estimated Cost to Date,Estimated Population,Estimated Housing Units
0,"Pool Hollow - Wildfire\n(Centre County, PA)\n",Centre,PA,"April 20, 2023, 2:38 p.m.",100%,"1,200","$175,000",295,130
1,"Buzby Boggs - Wildfire\n(Burlington County, NJ)\n",Burlington,NJ,"June 9, 2023, 8:30 p.m.",100%,700,"$6,000","11,804","4,640"
2,"Box Turtle - Wildfire\n(Gloucester County, NJ)\n",Gloucester,NJ,"May 28, 2023, 6:30 p.m.",100%,158,"$2,500","1,335",653
3,"Dragway - Wildfire\n(Camden County, NJ)\n",Camden,NJ,"Aug. 20, 2023, 2:45 p.m.",100%,"1,500","$9,000","2,003",765
4,"Acorn Hill - Wildfire\n(Burlington County, NJ)\n",Burlington,NJ,"June 19, 2023, 11 a.m.",100%,246,"$5,000",86,33
5,"City Line - Wildfire\n(Burlington County, NJ)\n",Burlington,NJ,"June 9, 2023, 11:30 a.m.",100%,850,"$6,000",46,18
6,"RUSH CREEK - Wildfire\n(Allegany County, NY)\n",Allegany,NY,"Nov. 16, 2023, 5:15 p.m.",100%,138,"$120,000",361,679
7,"Air Park - Wildfire\n(Ocean County, NJ)\n",Ocean,NJ,"Sept. 2, 2023, 11:40 p.m.",100%,831,"$5,000",320,119
8,"Cannon Ball 2 - Wildfire\n(Passaic County, NJ)\n",Passaic,NJ,"May 13, 2023, 7:28 a.m.",100%,107,"$4,000","23,085","9,305"
9,"Five Mile - Wildfire\n(Kanawha County, WV)\n",Kanawha,WV,"Nov. 4, 2023, 6 p.m.",100%,300,$500,377,171


In [95]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://poweroutage.us/area/state/pennsylvania"

# Send a GET request to the URL
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

# Find the table
table = soup.find('table')

# Extract the rows from the table
rows = table.find_all('tr')

# Initialize empty lists to store data
data = []

# Iterate through the rows and extract the data
for row in rows[1:]:  # Start from the second row to skip the header row
    columns = row.find_all('div')
    provider = columns[0].text.strip()
    customers_tracked = columns[1].text.strip().replace(',', '')  # Remove comma in numbers
    customers_out = columns[2].text.strip().replace(',', '')  # Remove comma in numbers
    last_updated = columns[3].text.strip()

    data.append([provider, customers_tracked, customers_out, last_updated])

# Create a DataFrame from the extracted data
df = pd.DataFrame(data, columns=['Provider', 'Customers Tracked', 'Customers Out', 'Last Updated'])

# Display the DataFrame
print(df)

                               Provider Customers Tracked Customers Out  \
0            Adams Electric Cooperative             34224             0   
1          Central Electric Cooperative             25548             0   
2                    Citizens' Electric              7640             0   
3                         Claverack REC             23352             0   
4                Duquesne Light Company            588000             9   
5                           FirstEnergy           2036277          1681   
6                     Northwestern RECA             20825             0   
7         Philadelphia Electric Company           1663160           355   
8                PPL Electric Utilities           1891083           120   
9                            REA Energy             25647             0   
10  Somerset Rural Electric Cooperative             13903             0   
11                  Sullivan County REC              6169             1   
12       Tri-County Rural

In [98]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://data.lansingstatejournal.com/national-power-outage-map-tracker/area/pennsylvania/42/"

# Send a GET request to the URL
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

# Find the table
table = soup.find('table')

# Extract the rows from the table
rows = table.find_all('tr')

columns = ', '.join(str(rows[0]).split("</th><th>")).replace("<tr><th>","").replace("</th></tr>","").split(", ")
final_df = pd.DataFrame(columns = columns)
index=0

for i in rows[1:]:
  test = str(i).replace("""<tr><td><a href="/national-power-outage-map-tracker/area/""","").replace("</th><th>"," ").replace("</th></tr>"," ").replace("<tr><th>"," ").replace("""class="ar">"""," ").replace("</td><td","").replace("</td> </tr>","").split("</a>")#.split(">")[1].strip().split(" ")
  final_df.at[index,'Area'] = test[0]
  #test[1] = test[1].lstrip()
  metrics = test[1].split(' ')
  final_df.at[index,'% of power outage'] = metrics[2]
  final_df.at[index,'Number of outages reported'] = metrics[4]
  final_df.at[index,'Number of customers tracked'] = metrics[6]
  #master_data[index] = data
  index+=1

final_df['Area'] = final_df['Area'].apply(lambda x: x.split(">")[1])
final_df

,Area,% of power outage,Number of outages reported,Number of customers tracked
0,Pennsylvania,0.0,"2,149","6,820,517"
1,"Huntingdon County, PA",5.6,760,"13,564"
2,"Jefferson County, PA",2.4,587,"24,352"
3,"Armstrong County, PA",0.3,127,"37,042"
4,"Clarion County, PA",0.2,42,"22,425"
5,"Clearfield County, PA",0.1,40,"42,625"
6,"Allegheny County, PA",0.0,13,"896,211"
7,"Berks County, PA",0.0,11,"193,926"
8,"Blair County, PA",0.0,9,"55,777"
9,"Bucks County, PA",0.0,66,"268,384"


In [156]:
url = "https://www.fireweatheravalanche.org/wildfires/dashboard/pennsylvania"

# Send a GET request to the URL
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

soup

no_drought = soup.find_all("div", {"class": "drought d-1"})
abnormally_dry = soup.find_all("div", {"class": "drought d-1"})
moderately_dry = soup.find_all("div", {"class": "drought d1"})
current_wildfires = soup.find_all("div", {"class": "col w100 center"})

current_stats = str(soup.find_all("div", {"class": "cur-stats"})[0]).split(">")
current_stats
total_number_of_large_fires = current_stats[4]
new_fires = current_stats[12]
year_to_date_fires = current_stats[20]
year_to_date_fires

current_stats_two = str(soup.find_all("div", {"class": "cur-stats"})[1]).split(">")
acres_burned_on_large_fires = current_stats_two[4]
personnel_assigned_to_wildfire = current_stats_two[12]
year_to_date_acres_burned = current_stats_two[20]

current_stats_three = str(soup.find_all("div", {"class": "cur-stats"})[2]).split(">")
average_acres_per_fire = current_stats_three[4]
large_fires_to_all_fires = current_stats_three[12]
average_acres_per_minute = current_stats_three[20]
average_acres_per_minute

'5.3</h2'